In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:


from config import filenames, folders

from matplotlib import pyplot as plt

device = 'cpu'

config = {
    'model_name': 'LSTM_3',
    'feature': 'max-regularization-fine-tune',
    'max_len': 42,
    'min_freq_src': 5,
    'min_freq_trg': 5,

    'embedding_dim': 120,
    'hidden_size': 360,
    'num_layers': 3,

    'num_epochs': 15,
    'weight_decay': 2e-5,
    'label_smoothing': 0.1,

    'dropout_emb': 0.25,

    'dropout_enc': 0.5,
    'dropout_dec': 0.5,

    'dropout_attention': 0.15,

    'learning_rate': 3e-4,
    'gamma': 0.2,
    'patience': 1,
    'threshold': 2e-4,
    'batch_size': 128,

    'use_tf': False,
    'tf_from_epoch': 0,
    'tf_start': 0.9,
    'tf_decrease': 0.02
}

config['weights'] = '../weights/saves/lstm-save-15.pt'

def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Training Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss Over Epochs")
    plt.legend()
    plt.grid()
    plt.show()

In [60]:
from dataset import Vocab
vocab_src = Vocab(filenames['train_src'], min_freq=config['min_freq_src'])
vocab_trg = Vocab(filenames['train_trg'], min_freq=config['min_freq_trg'])

In [61]:
print(len(vocab_src))
print(len(vocab_trg))
config['src_vocab_size'] = len(vocab_src)
config['trg_vocab_size'] = len(vocab_trg)

24991
18710


In [184]:
from dataset import TranslationDataset
train_dataset = TranslationDataset(vocab_src, 
                                vocab_trg, 
                                filenames['train_src'], 
                                filenames['train_trg'], 
                                max_len=config['max_len'], 
                                device=device)
val_dataset = TranslationDataset(vocab_src, 
                                vocab_trg, 
                                filenames['test_src'], 
                                filenames['test_trg'], 
                                max_len=72, 
                                device=device, 
                                sort_lengths=False)


100%|██████████| 986/986 [00:00<00:00, 15234.99it/s]


In [63]:
from lstm3 import train, LSTM_3
model = LSTM_3(config=config)

In [190]:
from submission import get_bleu
from dataset import TestDataLoader, RawDataset

val_loader = TestDataLoader(val_dataset, batch_size=256)

raw = RawDataset(filenames['test_src'])

get_bleu(model, val_loader, vocab_trg, device=device, use_beam=0, border=0.00/100, beam_width=1, raw_dataset=raw)

 75%|███████▌  | 3/4 [00:17<00:05,  5.80s/it]

correct:	 2657 244 42 9
total:		 17651 16665 15679 14701
sinergy:	0.78
BP:		0.864
15.1/1.5/0.3/0.1


np.float64(0.67)

In [ ]:
for src, trg in val_loader:
    print(vocab_trg.decode(trg[0]))
    model.inference(src, verbose=0, max_len=18, vocab_trg=vocab_trg, vocab_src=vocab_src, device=device, beam_width=3)
    res = model.inference_beam(src, verbose=0, max_len=18, vocab_trg=vocab_trg, vocab_src=vocab_src, device=device, beam_width=1, border=0.01)
    print(vocab_trg.decode(res[0]))
    break
# print('over')

['when', 'i', 'was', '<NUM>', ',', 'i', 'remember', 'waking', 'up', 'one', 'morning', 'to', 'the', 'sound', 'of', 'joy', 'in', 'my', 'house', '.']
['when', 'i', 'was', '<NUM>', 'years', 'old', ',', 'i', 'was', 'given', 'by', 'the', 'sound']


In [141]:
import torch
a = torch.tensor([[1, 2, 3]])
s = a.size()
b = torch.tensor([[0, 0, 3]])
print(a.size(), b.size())
a = a.masked_select((b == 2), )
a.view(s)

torch.Size([1, 3]) torch.Size([1, 3])


RuntimeError: shape '[1, 3]' is invalid for input of size 0